In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - fit_ossipkov_merrit_df.ipynb
# AUTHOR - James Lane
# PROJECT - sample_project
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Try and do some fits with Ossipkov-Merrit distribution functions.
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, copy, glob, subprocess, warnings, dill as pickle

## Matplotlib
import matplotlib as mpl
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu
from astropy.coordinates import SkyCoord

## galpy
# from galpy import orbit
# from galpy import potential
# from galpy import actionAngle as aA
# from galpy import df

## Project-specific
sys.path.insert(0,'../../src/')
import tng_dfs.kinematics as pkin

### Notebook setup

# %matplotlib inline
# plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
# %config InlineBackend.figure_format = 'retina'
# %load_ext autoreload
# %autoreload 2

## Make some plots of Ossipkov-Merrit DF anisotropy

### Standard anisotropy as a function of radius

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

rs = np.logspace(-2,2,num=1001)
ra = np.logspace(-1,1,num=10)

for i in range(len(ra)):
    beta = pkin.beta_ossipkov_merrit(rs,ra=ra[i])
    ax.plot(rs, beta, color='Black')
    ax.axvline(ra[i], color='Black', linestyle='dashed')

# cbar = plt.colorbar()
ax.set_xlabel('r [kpc]')
ax.set_ylabel('beta')
ax.set_xscale('log')

fig.show()

### Flexible anisotropy as a function of radius

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

rs = np.logspace(-2,2,num=1001)
ra = 1
alpha = np.linspace(-0.5,0.5,num=10)

for i in range(len(alpha)):
    beta = pkin.beta_any_alpha_cuddeford91(rs,ra=ra,alpha=alpha[i])
    ax.plot(rs, beta, color='Black')
    ax.axhline(alpha[i], color='Black', linestyle='dashed')

ax.set_xlabel('r [kpc]')
ax.set_ylabel('beta')
ax.set_xscale('log')
ax.axvline(ra, color='Black', linestyle='dotted')

fig.show()

### Load some data and try and do some fits at $z=0$

What likelihood to use? Perhaps just use weighted least squares on the binned 
data to begin with


Then compute the reduced chi square statistic on the binned data.

$\Chi^{2}_{\nu} = \Chi^{2}/\nu$
$\nu = m - n$


$n$ is the number of observations, $m$ is the number of fitted parameters